# 第六章 评估

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、 LLM生成测试用例](#二、-LLM生成测试用例)
     - [2.1 创建LLM应用](#2.1-创建LLM应用)
     - [2.2 通过LLM生成测试用例](#2.2-通过LLM生成测试用例)
 - [三、 人工评估](#三、-人工评估)
     - [3.1 如何评估新创建的实例](#3.1-如何评估新创建的实例)
     - [3.2 中文版](#3.2-中文版)
 - [四、 通过LLM进行评估实例](#四、-通过LLM进行评估实例)
     - [4.1  评估思路](#4.1--评估思路)
     - [4.2 结果分析](#4.2-结果分析)
     - [3.3 通过LLM进行评估实例](#3.3-通过LLM进行评估实例)


## 一、设置OpenAI API Key

登陆 [OpenAI 账户](https://platform.openai.com/account/api-keys) 获取API Key，然后将其设置为环境变量。

- 如果你想要设置为全局环境变量，可以参考[知乎文章](https://zhuanlan.zhihu.com/p/627665725)。
- 如果你想要设置为本地/项目环境变量，在本文件目录下创建`.env`文件, 打开文件输入以下内容。

    <p style="font-family:verdana; font-size:12px;color:green">
    OPENAI_API_KEY="your_api_key"
    </p>

  替换"your_api_key"为你自己的 API Key

## 二、 LLM生成测试用例
### 2.1 创建LLM应用

In [1]:
#查看数据
import pandas as pd
file = 'data/product_data.csv'
data = pd.read_csv(file,header=None)
data.head()

,0,1
0,product_name,description
1,全自动咖啡机,规格:\n大型 - 尺寸：13.8'' x 17.3''。\n中型 - 尺寸：11.5'' ...
2,电动牙刷,规格:\n一般大小 - 高度：9.5''，宽度：1''。\n\n为什么我们热爱它:\n我们的...
3,橙味维生素C泡腾片,规格:\n每盒含有20片。\n\n为什么我们热爱它:\n我们的橙味维生素C泡腾片是快速补充维...
4,无线蓝牙耳机,规格:\n单个耳机尺寸：1.5'' x 1.3''。\n\n为什么我们热爱它:\n这款无线蓝...


In [2]:
import os
from langchain.chains.retrieval_qa.base import RetrievalQA #检索QA链，在文档上进行检索
from langchain_openai import ChatOpenAI #openai模型
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores.docarray.in_memory import DocArrayInMemorySearch


file = 'data/product_data.csv'
API_KEY = os.environ.get("CHAT_ANYWHERE_API_KEY")
BASE_URL = "https://api.chatanywhere.com.cn/v1"

#创建一个文档加载器，通过csv格式加载
loader = CSVLoader(file_path=file)
docs = loader.load()
print(docs[0])

# 向量
embeddings = OpenAIEmbeddings(openai_api_base=BASE_URL,
                              openai_api_key=API_KEY,
                              model= "text-embedding-ada-002") #初始化
print(type(embeddings))

#因为文档比较短了，所以这里不需要进行任何分块,可以直接进行向量表征
#使用初始化OpenAIEmbedding实例上的查询方法embed_query为文本创建向量表征
embed = embeddings.embed_query("Hi my name is Harrison")

# 基于向量表征创建向量存储
db = DocArrayInMemorySearch.from_documents(docs, embeddings)


llm = ChatOpenAI(base_url=BASE_URL, api_key=API_KEY,
                 model="gpt-3.5-turbo-0301", temperature=0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    verbose=True
)

qa

page_content="product_name: 全自动咖啡机\ndescription: 规格:\n大型 - 尺寸：13.8'' x 17.3''。\n中型 - 尺寸：11.5'' x 15.2''。\n\n为什么我们热爱它:\n这款全自动咖啡机是爱好者的理想选择。 一键操作，即可研磨豆子并沏制出您喜爱的咖啡。它的耐用性和一致性使它成为家庭和办公室的理想选择。\n\n材质与护理:\n清洁时只需轻擦。\n\n构造:\n由高品质不锈钢制成。\n\n其他特性:\n内置研磨器和滤网。\n预设多种咖啡模式。\n在中国制造。\n\n有问题？ 请随时联系我们的客户服务团队，他们会解答您的所有问题。" metadata={'source': 'data/product_data.csv', 'row': 0}
<class 'langchain_openai.embeddings.base.OpenAIEmbeddings'>


/Users/chubin.zheng/ENVS/llm-learning/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


RetrievalQA(verbose=True, combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x14cd7b9d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x14c4797e0>, model_name='gpt-3.5-turbo-0301', temperature=0.0, openai_api_key=SecretStr('**********'), openai_api_base='https://api.chatanywhere.com.cn/v1', openai_proxy='')), document_variable_name='context'), retriever=VectorStoreRetriever(tags=['DocArrayInMemorySearch'], v

### 2.1 通过LLM生成测试用例
1、将自己想出好的数据点作为例子，查看一些数据，然后想出例子问题和答案，以便以后用于评估

In [3]:
docs[10]

Document(page_content="product_name: 高清电视机\ndescription: 规格:\n尺寸：50''。\n\n为什么我们热爱它:\n我们的高清电视机拥有出色的画质和强大的音效，带来沉浸式的观看体验。\n\n材质与护理:\n使用干布清洁。\n\n构造:\n由塑料、金属和电子元件制成。\n\n其他特性:\n支持网络连接，可以在线观看视频。\n配备遥控器。\n在韩国制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。", metadata={'source': 'data/product_data.csv', 'row': 10})

In [35]:
# 测试用例数据
examples = [
    {
        "query": "高清电视机怎么进行护理？",
        "answer": "使用干布清洁。"
    },
    {
        "query": "旅行背包有内外袋吗？",
        "answer": "有。"
    }
]

2、通过LLM生成测试用例
由于`QAGenerateChain`类中使用的`PROMPT`是英文，故我们继承`QAGenerateChain`类，将`PROMPT`加上“请使用中文输出”。

In [40]:
from typing import Any
from langchain_core.language_models import BaseLanguageModel
from langchain.evaluation.qa import QAGenerateChain # QA生成链:接收文档，并从每个文档中创建一个问题答案对
from langchain.evaluation.qa.generate_prompt import template
from langchain_core.prompts import PromptTemplate


MY_TEMPLATE = template + "\n请使用中文输出."
MY_PROMPT = PromptTemplate(input_variables=["doc"], template=MY_TEMPLATE)


class MyQAGenerateChain(QAGenerateChain):
    """继承QAGenerateChain"""

    @classmethod
    def from_llm(cls, llm: BaseLanguageModel, **kwargs: Any) -> QAGenerateChain:
        """Load QA Generate Chain from LLM."""
        return cls(llm=llm, prompt=MY_PROMPT, **kwargs)


# 通过传递chat open AI语言模型来创建这个链
example_gen_chain = MyQAGenerateChain.from_llm(llm=llm)
new_examples = example_gen_chain.apply_and_parse([{"doc": t} for t in docs[:5]])
new_examples

/Users/chubin.zheng/ENVS/llm-learning/lib/python3.10/site-packages/langchain/chains/llm.py:367: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[{'qa_pairs': {'query': '这款全自动咖啡机的尺寸有哪些选择？ ',
   'answer': "这款全自动咖啡机有大型和中型两种尺寸，分别为13.8'' x 17.3''和11.5'' x 15.2''。"}},
 {'qa_pairs': {'query': '这款电动牙刷的材质是什么制成的？', 'answer': '这款电动牙刷由食品级塑料和尼龙刷毛制成。'}},
 {'qa_pairs': {'query': '这种泡腾片的主要成分是什么？', 'answer': '主要成分为维生素C和柠檬酸钠。'}},
 {'qa_pairs': {'query': '这款无线蓝牙耳机有哪些特点？',
   'answer': '这款无线蓝牙耳机配备了降噪技术和长达8小时的电池续航力，内置麦克风，支持接听电话，具有快速充电功能。它由耐用的塑料和金属构成，配备有软质耳塞，只需用湿布清洁。此外，这款耳机在韩国制造。'}},
 {'qa_pairs': {'query': '这款瑜伽垫的尺寸是多少？', 'answer': "这款瑜伽垫的尺寸是24'' x 68''。"}}]

In [41]:
new_examples = list(map(lambda x: x.pop('qa_pairs'), new_examples))
examples += new_examples
qa.invoke(examples[2]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


{'query': '这款全自动咖啡机的尺寸有哪些选择？ ',
 'result': "这款全自动咖啡机有两种尺寸可选：大型尺寸为13.8'' x 17.3''，中型尺寸为11.5'' x 15.2''。"}

## 三、 人工评估
现在有了这些示例，但是我们如何评估正在发生的事情呢？
通过运行一个示例通过链，并查看它产生的输出
在这里我们传递一个查询，然后我们得到一个答案。实际上正在发生的事情，进入语言模型的实际提示是什么？   
它检索的文档是什么？   
中间结果是什么？    
仅仅查看最终答案通常不足以了解链中出现了什么问题或可能出现了什么问题

In [17]:
''' 
LingChainDebug工具可以了解运行一个实例通过链中间所经历的步骤
'''
import langchain
langchain.debug = True
qa.run(examples[0]["query"])#重新运行与上面相同的示例，可以看到它开始打印出更多的信息

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "高清电视机怎么进行护理？"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "高清电视机怎么进行护理？",
  "context": "product_name: 高清电视机\ndescription: 规格:\n尺寸：50''。\n\n为什么我们热爱它:\n我们的高清电视机拥有出色的画质和强大的音效，带来沉浸式的观看体验。\n\n材质与护理:\n使用干布清洁。\n\n构造:\n由塑料、金属和电子元件制成。\n\n其他特性:\n支持网络连接，可以在线观看视频。\n配备遥控器。\n在韩国制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。\n\nproduct_name: 空气净化器\ndescription: 规格:\n尺寸：15'' x 15'' x 20''。\n\n为什么我们热爱它:\n我们的空气净化器采用了先进的HEPA过滤技术，能有效去除空气中的微粒和异味，为您提供清新的室内环境。\n\n材质与护理:\n清洁时使用干布擦拭。\n\n构造:\n由塑料和电子元件制成。\n\n其他特性:\n三档风速，附带定时功能。\n在德国制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。\n\nproduct_name: 宠物自动喂食器\ndescription: 规格:\n尺寸：14'' x 9'' x 15''。\n\n为什么我们热爱它:\n我们的宠物自动喂食器可以定时定量投放食物，让您无论在家或外出都能确保宠物的饮食。\n\n材质与护理:\n可用湿布清洁。\n\n构造:\n由塑料和电子元件制成。\n\n其他特性:\n配备LCD屏幕，操作简单。\n可以设置多次投食

'您可以使用干布清洁高清电视机。避免使用水或化学清洁剂，因为这些可能会损坏电视机的表面。'

我们可以看到它首先深入到检索QA链中，然后它进入了一些文档链。如上所述，我们正在使用stuff方法，现在我们正在传递这个上下文，可以看到，这个上下文是由我们检索到的不同文档创建的。因此，在进行问答时，当返回错误结果时，通常不是语言模型本身出错了，实际上是检索步骤出错了，仔细查看问题的确切内容和上下文可以帮助调试出错的原因。    
然后，我们可以再向下一级，看看进入语言模型的确切内容，以及 OpenAI 自身，在这里，我们可以看到传递的完整提示，我们有一个系统消息，有所使用的提示的描述，这是问题回答链使用的提示，我们可以看到提示打印出来，使用以下上下文片段回答用户的问题。
如果您不知道答案，只需说您不知道即可，不要试图编造答案。然后我们看到一堆之前插入的上下文，我们还可以看到有关实际返回类型的更多信息。我们不仅仅返回一个答案，还有token的使用情况，可以了解到token数的使用情况


由于这是一个相对简单的链，我们现在可以看到最终的响应，舒适的毛衣套装，条纹款，有侧袋，正在起泡，通过链返回给用户，我们刚刚讲解了如何查看和调试单个输入到该链的情况。




## 四、 通过LLM进行评估实例

In [42]:
langchain.debug = False
predictions = qa.batch(examples) #为所有不同的示例创建预测
predictions



> Entering new RetrievalQA chain...

> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


[{'query': '高清电视机怎么进行护理？',
  'answer': '使用干布清洁。',
  'result': '您可以使用干布清洁高清电视机。避免使用含有酒精、丙酮、氨水等化学物质的清洁剂，以免对电视机造成损害。'},
 {'query': '旅行背包有内外袋吗？',
  'answer': '有。',
  'result': '是的，我们的旅行背包拥有多个实用的内外袋，可以轻松装下您的必需品。'},
 {'query': '这款全自动咖啡机的尺寸有哪些选择？ ',
  'answer': "这款全自动咖啡机有大型和中型两种尺寸，分别为13.8'' x 17.3''和11.5'' x 15.2''。",
  'result': "这款全自动咖啡机有两种尺寸可选：大型尺寸为13.8'' x 17.3''，中型尺寸为11.5'' x 15.2''。"},
 {'query': '这款电动牙刷的材质是什么制成的？',
  'answer': '这款电动牙刷由食品级塑料和尼龙刷毛制成。',
  'result': '这款电动牙刷的构造是由食品级塑料和尼龙刷毛制成的。'},
 {'query': '这种泡腾片的主要成分是什么？',
  'answer': '主要成分为维生素C和柠檬酸钠。',
  'result': '这种泡腾片的主要成分是维生素C和柠檬酸钠。'},
 {'query': '这款无线蓝牙耳机有哪些特点？',
  'answer': '这款无线蓝牙耳机配备了降噪技术和长达8小时的电池续航力，内置麦克风，支持接听电话，具有快速充电功能。它由耐用的塑料和金属构成，配备有软质耳塞，只需用湿布清洁。此外，这款耳机在韩国制造。',
  'result': '这款无线蓝牙耳机有以下特点：\n- 配备了降噪技术，可以减少外界噪音的干扰，提供更好的音乐体验。\n- 电池续航力长达8小时，可以让您在任何地方都能享受无障碍的音乐体验。\n- 由耐用的塑料和金属构成，配备有软质耳塞，舒适度高。\n- 支持快速充电功能和内置麦克风，可以接听电话。\n- 制造商在韩国。'},
 {'query': '这款瑜伽垫的尺寸是多少？',
  'answer': "这款瑜伽垫的尺寸是24'' x 68''。",
  'result': "有两款瑜伽垫，第一款的尺寸是24'' x 68

In [43]:
''' 
对预测的结果进行评估，导入QA问题回答，评估链，通过语言模型创建此链
'''
from langchain.evaluation.qa import QAEvalChain #导入QA问题回答，评估链


eval_chain = QAEvalChain.from_llm(llm)  #通过调用chatGPT进行评估
graded_outputs = eval_chain.evaluate(examples, predictions)  #在此链上调用evaluate，进行评估

#### 评估思路
当它面前有整个文档时，它可以生成一个真实的答案，我们将打印出预测的答，当它进行QA链时，使用embedding和向量数据库进行检索时，将其传递到语言模型中，然后尝试猜测预测的答案，我们还将打印出成绩，这也是语言模型生成的。当它要求评估链评估正在发生的事情时，以及它是否正确或不正确。因此，当我们循环遍历所有这些示例并将它们打印出来时，可以详细了解每个示例

In [46]:
#我们将传入示例和预测，得到一堆分级输出，循环遍历它们打印答案
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: 高清电视机怎么进行护理？
Real Answer: 使用干布清洁。
Predicted Answer: 您可以使用干布清洁高清电视机。避免使用含有酒精、丙酮、氨水等化学物质的清洁剂，以免对电视机造成损害。
Predicted Grade: CORRECT

Example 1:
Question: 旅行背包有内外袋吗？
Real Answer: 有。
Predicted Answer: 是的，我们的旅行背包拥有多个实用的内外袋，可以轻松装下您的必需品。
Predicted Grade: CORRECT

Example 2:
Question: 这款全自动咖啡机的尺寸有哪些选择？ 
Real Answer: 这款全自动咖啡机有大型和中型两种尺寸，分别为13.8'' x 17.3''和11.5'' x 15.2''。
Predicted Answer: 这款全自动咖啡机有两种尺寸可选：大型尺寸为13.8'' x 17.3''，中型尺寸为11.5'' x 15.2''。
Predicted Grade: CORRECT

Example 3:
Question: 这款电动牙刷的材质是什么制成的？
Real Answer: 这款电动牙刷由食品级塑料和尼龙刷毛制成。
Predicted Answer: 这款电动牙刷的构造是由食品级塑料和尼龙刷毛制成的。
Predicted Grade: CORRECT

Example 4:
Question: 这种泡腾片的主要成分是什么？
Real Answer: 主要成分为维生素C和柠檬酸钠。
Predicted Answer: 这种泡腾片的主要成分是维生素C和柠檬酸钠。
Predicted Grade: CORRECT

Example 5:
Question: 这款无线蓝牙耳机有哪些特点？
Real Answer: 这款无线蓝牙耳机配备了降噪技术和长达8小时的电池续航力，内置麦克风，支持接听电话，具有快速充电功能。它由耐用的塑料和金属构成，配备有软质耳塞，只需用湿布清洁。此外，这款耳机在韩国制造。
Predicted Answer: 这款无线蓝牙耳机有以下特点：
- 配备了降噪技术，可以减少外界噪音的干扰，提供更好的音乐体验。
- 电池续航力长达8小时，可以让您在任何地方都能享受无障碍的音乐

#### 结果分析
对于每个示例，它看起来都是正确的，让我们看看第一个例子。
这里的问题是，旅行背包有内外袋吗？真正的答案，我们创建了这个，是肯定的。模型预测的答案是是的，旅行背包有多个实用的内外袋，可以轻松装下您的必需品。因此，我们可以理解这是一个正确的答案。它将其评为正确。    
#### 使用模型评估的优势

你有这些答案，它们是任意的字符串。没有单一的真实字符串是最好的可能答案，有许多不同的变体，只要它们具有相同的语义，它们应该被评为相似。如果使用正则进行精准匹配就会丢失语义信息，到目前为止存在的许多评估指标都不够好。目前最有趣和最受欢迎的之一就是使用语言模型进行评估。